In [15]:
import threading

# Helper thread to avoid the Spark StreamingContext from blocking Jupyter
        
class StreamingThread(threading.Thread):
    def __init__(self, ssc):
        super().__init__()
        self.ssc = ssc
    def run(self):
        self.ssc.start()
        self.ssc.awaitTermination()
    def stop(self):
        print('----- Stopping... this may take a few seconds -----')
        self.ssc.stop(stopSparkContext=False, stopGraceFully=True)

In [16]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [17]:
spark

In [18]:
socketDF = spark.readStream.format("socket").option("host", "seppe.net").option("port", 7778).load()
socketDF.printSchema()

root
 |-- value: string (nullable = true)



In [19]:
from pyspark.sql.functions import from_json, schema_of_json

In [20]:
def process_row(df, epoch_id):
    print(epoch_id)
    if df.count() == 0: return
    schema = schema_of_json(df.first().value)
    df_cols = df.selectExpr('CAST(value AS STRING)')\
        .select(from_json('value', schema)\
        .alias('temp'))\
        .select('temp.*')
    df_cols.show()
    # We can also save here using something such as:
    df.write.format("json").mode("append").save("data")

In [21]:
query = socketDF.writeStream.trigger(processingTime='30 seconds').foreachBatch(process_row).start()

0
1
+--------+--------+---------------+---------+-------------------+--------------------+-----------------+--------------------+--------------------+---------+-----+
|     aid|comments|         domain|frontpage|          posted_at|         source_text|     source_title|               title|                 url|     user|votes|
+--------+--------+---------------+---------+-------------------+--------------------+-----------------+--------------------+--------------------+---------+-----+
|40230217|       0|    astrid.tech|    false|2024-05-01 22:12:45|Ansible is a Lisp...|Ansible is a Lisp|   Ansible Is a Lisp|https://astrid.te...|      zdw|    1|
|40230259|       0|venturebeat.com|    false|2024-05-01 22:16:07|Just a moment...\...| Just a moment...|Unity announces a...|https://venturebe...|doomlaser|    2|
|40230274|       0|         ft.com|    false|2024-05-01 22:17:29|Satellite group S...|             NULL|Satellite group S...|https://www.ft.co...|bookofjoe|    1|
+--------+--------

In [8]:
query.stop()

0
